In [2]:
# -----------------------------------------------------------------------------
# 6. ENTRENAMIENTO DE MODELOS (LSTM, GRU, BiLSTM)
# -----------------------------------------------------------------------------
histories = {}

configs = [
    {'name': 'lstm_base', 'type': 'lstm', 'bi': False, 'attn': False},
    {'name': 'lstm_attn', 'type': 'lstm', 'bi': False, 'attn': True},
    {'name': 'gru_base',  'type': 'gru',  'bi': False, 'attn': False},
    {'name': 'gru_attn',  'type': 'gru',  'bi': False, 'attn': True},
    {'name': 'lstm_bi',   'type': 'lstm', 'bi': True,  'attn': False},
    {'name': 'lstm_bi_attn','type':'lstm','bi': True,  'attn': True}
]

for conf in configs:
    m_name = conf['name']
    m_type = conf['type']
    bi = conf['bi']
    attn = conf['attn']
    
    print(f"\n{'='*40}")
    print(f" Entrenando: {m_name.upper()} (Bi={bi}, Attn={attn})")
    print(f"{'='*40}")
    
    model = RecurrentClassifier(
        model_type=m_type,
        vocab_size=len(vocab),
        embed_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        out_dim=3,
        n_layers=N_LAYERS,
        dropout=DROPOUT,
        pad_idx=vocab["<PAD>"],
        bidirectional=bi,
        use_attention=attn
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
    
    best_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'total_time': 0}
    
    start_time = time.time() # Start timer
    
    for ep in range(EPOCHS):
        tl, ta = train_epoch(model, train_loader, optimizer, criterion)
        vl, va = evaluate(model, val_loader, criterion)
        scheduler.step()
        
        history['train_loss'].append(tl)
        history['val_loss'].append(vl)
        history['train_acc'].append(ta)
        history['val_acc'].append(va)
        
        print(f"Epoch {ep+1}/{EPOCHS} | Train Loss: {tl:.4f} Acc: {ta:.4f} | Val Loss: {vl:.4f} Acc: {va:.4f}")
        
        if va > best_acc:
            best_acc = va
            state = {
                'model_state': model.state_dict(),
                'config': {
                    'model_type': m_type,
                    'vocab_size': len(vocab),
                    'embed_dim': EMBEDDING_DIM, 
                    'hidden_dim': HIDDEN_DIM,
                    'n_layers': N_LAYERS,
                    'dropout': DROPOUT,
                    'pad_idx': vocab["<PAD>"],
                    'bidirectional': bi,
                    'use_attention': attn
                },
                'vocab': vocab
            }
            save_path = f"{MODELS_DIR}/{m_name}_best_model.pth"
            torch.save(state, save_path)
            print(f" --> Nuevo récord! Modelo guardado en {save_path}")
            
    end_time = time.time() # End timer
    total_time = end_time - start_time
    history['total_time'] = total_time
    print(f"Tiempo total de entrenamiento: {total_time:.2f} segundos")
    
    histories[m_name] = history

print("\n¡Entrenamiento finalizado para LSTM, GRU y BiLSTM!")

# Guardar historial completo
with open(f"{MODELS_DIR}/histories.pkl", "wb") as f:
    pickle.dump(histories, f)
print(f"Historial guardado en {MODELS_DIR}/histories.pkl")


Usando dispositivo: cpu
Cargando datos procesados...
Dataloaders listos.
Pesos por clase: [0.50463279 0.37781801 0.11754921]

 Entrenando: LSTM_BASE (Bi=False, Attn=False)
Epoch 1/12 | Train Loss: 1.0856 Acc: 0.4165 | Val Loss: 1.0715 Acc: 0.5958
 --> Nuevo récord! Modelo guardado en models/lstm_base_best_model.pth
Epoch 2/12 | Train Loss: 0.9693 Acc: 0.6325 | Val Loss: 0.9731 Acc: 0.6580
 --> Nuevo récord! Modelo guardado en models/lstm_base_best_model.pth
Epoch 3/12 | Train Loss: 0.8476 Acc: 0.6840 | Val Loss: 1.0869 Acc: 0.7091
 --> Nuevo récord! Modelo guardado en models/lstm_base_best_model.pth
Epoch 4/12 | Train Loss: 0.7135 Acc: 0.7658 | Val Loss: 0.9305 Acc: 0.6846
Epoch 5/12 | Train Loss: 0.6143 Acc: 0.8064 | Val Loss: 0.8833 Acc: 0.6972
Epoch 6/12 | Train Loss: 0.5251 Acc: 0.8396 | Val Loss: 1.0344 Acc: 0.7084
Epoch 7/12 | Train Loss: 0.4357 Acc: 0.8691 | Val Loss: 1.0388 Acc: 0.7203
 --> Nuevo récord! Modelo guardado en models/lstm_base_best_model.pth
Epoch 8/12 | Train Loss